# Wildfire - All Resolution Aggregation
A previous notebook `Wildfire-5Min-Aggregation` showed how we can aggregate the fires to a 5-minute (1/12 degree) spatial resolution. However, with the climatology, there are 3 other resolutions we can deal with: 10-minute (1/6 degree), 2.5-minute (1/24 degree), and 30-second (1/120 degree). We will create fire datasets for each of these, and write them to our drive. We will also cut-down the climatology data into the bounding box we care about.

## Import Packages
Like before, we need to install `rasterio`. We also need to mount our drive so we can access its files.

In [ ]:
!pip install rasterio

     |████████████████████████████████| 18.1MB 196kB/s 


In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import rasterio as rio
from rasterio.windows import Window

import os
import sys
import datetime as dt
from itertools import product
import time

import matplotlib
import matplotlib.pyplot as plt

import pprint

from google.colab import drive

import sqlite3

%matplotlib inline

In [ ]:
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


## Info before We get Started - PLEASE READ
Since the climatology is split up by month, we'll split up the fire data by month as well. The previous notebook had a 3-dimensional array, but being consistent with the format is important.

To save the number of files we write, the fires and climatology (for each metric) will be in a *single* `.npz` file. An `.npz` file is basically a way to save multiple `numpy` arrays in a single file, along with labels. Our labels will be the 3-letter abbreviations for the months (starting with a capital letter i.e. "Jan", "Feb", ...).

Unfortunately, masked arrays are not supported with this method, so instead, we save the original data (with the `no_data` value filled in) and save the mask separately in a `mask.npy` file. Since the fires and climatology are all the same resolution, we only need one mask file.

The directory structure will be `Preprocessed/{Resolution}` inside our `Data` folder.

To recap, each resolution folder, will have 9 files:

- `fires.npz` - Data for fires for each month in "Jan", "Feb", ... labels.
- `tmin.npz` - Minimum temperature (&deg;C)
- `tmax.npz` - Maximum temperature (&deg;C)
- `tavg.npz` - Average temperature (&deg;C)
- `prec.npz` - Precipitation (mm)
- `srad.npz` - Solar radiation (kJ m<sup>-2</sup> day<sup>-1</sup>)
- `wind.npz` - Wind speed (m/s)
- `vapr.npz` - Water vapor pressure (kPa)
- `mask.npy` - The mask 

Let's get started! As an extra pre-caution, if you need to delete anything from drive, visit Drive directly and delete it...it's dangerous to try to delete things through code. Don't accidently delete your entire drive!

## Metadata
General stuff to store, like filepaths, resolutions, metrics, etc.

In [ ]:
GDRIVEROOT = '/gdrive/My Drive/Data Science and Public Policy/Final project'
PREFIX = 'Data/Preprocessed'
metrics = ['tmin', 'tmax', 'tavg', 'prec', 
           'srad', 'wind', 'vapr']
resolutions = ['5m']
divisionsPerDegree = [12]
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Bounding box
LATTOP = 50
LATBOT = 24.5
LONLEFT = -125
LONRIGHT = -66.5

# The query that we'll run against the fire database...
query = """
  SELECT FIRE_YEAR, DISCOVERY_DOY, STATE, LATITUDE, LONGITUDE, FIRE_SIZE_CLASS
  FROM Fires
"""

# Method to convert the lat/longitude value
# to an index corresponding to our climatology
# data. See previous notebook for details...
def convertToClimateIndex(value, divisionsPerDegree, direction):
  # Start at -180, positive difference
  if direction == 'lon':
    if value < -180 or value >= 180:
      raise ValueError('value was out of range.')
    return int((value + 180) * divisionsPerDegree)
  # Start at 90, negative difference
  else:
    if value < -90 or value >= 90:
      raise ValueError('value was out of range.')
    return int((value - 90) * divisionsPerDegree * -1)

# Same method, but for converting our point-locations
# to a fire index...
def convertToFireIndex(value, divisionsPerDegree, direction):
  # Start at -180, positive difference
  if direction == 'lon':
    if value < -125 or value >= -66.5:
      raise ValueError('value was out of range.')
    return int((value + 125) * divisionsPerDegree)
  # Start at 90, negative difference
  else:
    if value <= 24.5 or value > 50:
      raise ValueError('value was out of range.')
    return int((value - 50) * divisionsPerDegree * -1)

# List of tuples corresponding to
# climatology indices...
climateIndices = [(
    convertToClimateIndex(LATTOP, divisions, 'lat'),
    convertToClimateIndex(LATBOT, divisions, 'lat'),
    convertToClimateIndex(LONLEFT, divisions, 'lon'),
    convertToClimateIndex(LONRIGHT, divisions, 'lon')
) for divisions in divisionsPerDegree]

# Make the directories if not already present...
for res in resolutions:
  os.makedirs(os.path.join(GDRIVEROOT, PREFIX, res), exist_ok=True)

Now that setup is done, we can start writing files. We can technically do this in one **giant** loop, but that might be hard to digest. Instead, we'll split it up. 

## Writing Climatology Files
The function `itertools.product` allows us to do the Cartesian product of the resolution and metric like on the site. We also need the divisions per degree to convert the values to indices. However, the resolutions, divisions, and indices need to be paired up one-to-one, and so we call `zip` first.

In [ ]:
start = time.perf_counter()
# By surrounding with parentheses, we can
# unpack the variables on the fly.
for (res, divisions, (LATTOP_index, LATBOT_index, LONLEFT_index, LONRIGHT_index)), metric in \
    product(zip(resolutions, divisionsPerDegree, climateIndices), metrics):

  print('-{} {}'.format(res, metric.upper()))
  localFolder = 'wc2.1_{}_{}'.format(res, metric)
  downloadSite = 'http://biogeo.ucdavis.edu/data/worldclim/v2.1/base/{}.zip'.format(localFolder)
  print('  - Downloading {}...'.format(localFolder + '.zip'), end='')
  # Download zip file
  !wget -q -N {downloadSite}
  print('Done')
  # Unzip to directory (-d),
  # don't show messages (-q),
  # and force overwrite (-o)
  print('  - Unzipping...', end='')
  !unzip -q -o {localFolder}.zip -d ./{localFolder}
  print('Done')
  # Remove the zip file
  # !rm {localFolder}.zip

  # Create a dictionary that will hold the
  # data for each month. Eventually we will unpack
  # this to save in a compressed npz file.
  metricDict = {}
  # Files are numbered 01 to 12...
  print('  - Reading months...', end='')
  for i, month in enumerate(months, 1):
    tifFile = '{}_{:02}.tif'.format(localFolder, i)
    with rio.open(os.path.join(localFolder, tifFile)) as data:
      # Windowed reading...
      width = LONRIGHT_index - LONLEFT_index
      height = LATBOT_index - LATTOP_index
      dataArray = data.read(1, window=Window(col_off=LONLEFT_index, row_off=LATTOP_index,
                                             width=width, height=height))
      noDataVal = data.nodata
      dataMask = ~data.dataset_mask().astype(bool)[LATTOP_index : LATBOT_index, LONLEFT_index : LONRIGHT_index]
    # Create masked array object...
    # data = ma.core.MaskedArray(data=dataArray, mask=dataMask, 
    #                             fill_value=noDataVal)
    # Subset to USA and assign it to our dictionary
    # under the appropriate month...
    # usaData = data[LATTOP_index : LATBOT_index, LONLEFT_index : LONRIGHT_index]
    # usaMask = usaData.mask
    metricDict[month] = dataArray
  print('Done')
  
  # SAVE THIS METRIC!
  print('  - Uploading {}...'.format(metric), end='')
  np.savez(os.path.join(GDRIVEROOT, PREFIX, res, '{}.npz'.format(metric)),
            **metricDict)
  print('Done')

  # ONLY SAVE THE MASK IF WE HAVEN'T!
  maskFilePath = os.path.join(GDRIVEROOT, PREFIX, res, 'mask.npy')
  # if not os.path.exists(maskFilePath):
  print('  - Uploading mask...', end='')
  np.save(maskFilePath, dataMask)
  print('Done')

end = time.perf_counter()
print('FINISHED IN', end - start, 'SECONDS.')

-5m TMIN
  - Downloading wc2.1_5m_tmin.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploading tmin...Done
  - Uploading mask...Done
-5m TMAX
  - Downloading wc2.1_5m_tmax.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploading tmax...Done
  - Uploading mask...Done
-5m TAVG
  - Downloading wc2.1_5m_tavg.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploading tavg...Done
  - Uploading mask...Done
-5m PREC
  - Downloading wc2.1_5m_prec.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploading prec...Done
  - Uploading mask...Done
-5m SRAD
  - Downloading wc2.1_5m_srad.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploading srad...Done
  - Uploading mask...Done
-5m WIND
  - Downloading wc2.1_5m_wind.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploading wind...Done
  - Uploading mask...Done
-5m VAPR
  - Downloading wc2.1_5m_vapr.zip...Done
  - Unzipping...Done
  - Reading months...Done
  - Uploa

In [ ]:
maskFromFile = np.load(maskFilePath)

In [ ]:
np.array_equal(dataMask, maskFromFile)

True

In [ ]:
maskFilePath

'/gdrive/My Drive/Data Science and Public Policy/Final project/Data/Preprocessed/5m/mask.npy'

## Writing Fire Files...
Now we can write the fire files. Here, there wil only be 1 file per resolutions. In the previous notebook, we made sure that the fire data had the same shape. However, we can calculate the shape easily using the latitude longitude bounds and the number of divisions per degree. 

Additionally, because the query wil be the same, by separating the two loops, we only have to go through our fire database once. Each fire we will calculate the correct indices for each of the 4 resolutions, and then move onto the next fire.

As a reminder, we are skipping fires in Alaska, Hawaii, and Puerto Rico.

We've already written the masks in the previous loops so we can just read them directly here...

In [ ]:
dbPrefix = 'Data/RDS-2013-0009.4_SQLITE/Data'
dbFile = 'FPA_FOD_20170508.sqlite'
# Establish "connection" to the database
connection = sqlite3.connect(os.path.join(GDRIVEROOT, dbPrefix, dbFile))

In [ ]:
# Make a cursor...
cursor = connection.cursor()

start = time.perf_counter()

# First create the 4 fire datasets...
fireDatasets = []
for res, divisions in zip(resolutions, divisionsPerDegree):
  # Calculate the size of this fire dataset...
  height = int((LATTOP - LATBOT) * divisions)
  width = int((LONRIGHT - LONLEFT) * divisions)
  print(width, height)
  # Create a 3d array...
  fireDatasets.append(np.zeros(shape=(12, height, width), dtype=np.int32))

# NOW EXECUTE THE QUERY...
# Copy-pasted from the old notebook...
for row in cursor.execute(query):
  # Extract information
  year, doy, state, lat, lon, sizeClass = row
  # If the state is Alaska (AK) or
  # Hawaii (HI) or Puerto Rico (PR), then skip it...
  if state in ['AK', 'HI', 'PR'] or sizeClass in ['A', 'B']:
    continue
  # Create a date object for the first day of the year,
  # and add the proper number of days according to doy.
  # These are 1-indexed so we add one less.
  # Then, grab the month...which is also 1-indexed so we'll subtract
  # 1 when incrementing the fire count at that location.
  # Leap years are already taken into account with the package....
  fireMonth = (dt.date(year, 1, 1) + dt.timedelta(days=doy - 1)).month

  # Now, for each of the fire datasets,
  # calculate the correct latitude and longitude,
  # and increment the fire count at that location...
  # Rows correspond to latitudes,
  # columns correspond to longitudes...
  for i, divisions in enumerate(divisionsPerDegree):
    latIndex = convertToFireIndex(lat, divisions, 'lat')
    lonIndex = convertToFireIndex(lon, divisions, 'lon')
    # Increment the fire count!!
    fireDatasets[i][fireMonth - 1, latIndex, lonIndex] += 1

# Now, last step is to apply the same mask as
# the climatology and remove some values,
# then we save it a npz files...
for i, res in enumerate(resolutions):
  resMask = np.load(os.path.join(GDRIVEROOT, PREFIX, res, 'mask.npy'))
  fullMask = np.asarray([np.copy(resMask) for _ in range(12)])
  fireDatasets[i] = ma.core.MaskedArray(fireDatasets[i], mask=fullMask)
  # Create dictionary for each month...
  fireDict = {month: fireDatasets[i].data[j] for j, month in enumerate(months)}
  # SAVE THE FIRE...
  np.savez(os.path.join(GDRIVEROOT, PREFIX, res, 'firesC+.npz'), **fireDict)

end = time.perf_counter()
print('Finished in', end - start, 'seconds.')

702 306
Finished in 8.555980020000106 seconds.


In [ ]:
firesAplus = np.load(os.path.join(GDRIVEROOT, PREFIX, '5m', 'fires.npz'))
np.sum(firesAplus['Jan'])

92852

In [ ]:
np.sum(fireDict['Jan'])

16122

In [ ]:
print(sum([np.sum(firesAplus[month]) for month in months]))
print(sum([np.sum(fireDict[month]) for month in months]))

1835646
268042


In [ ]:
fireAplusJan = np.copy(firesAplus['Jan'])
fireCplusJan = np.copy(fireDict['Jan'])

In [ ]:
fireAplusJan[fireAplusJan > 0] = 1
fireCplusJan[fireCplusJan > 0] = 1

In [ ]:
np.mean(fireAplusJan)

0.11035696329813977

In [ ]:
np.mean(fireCplusJan)

0.042735042735042736